In [ ]:
!nvidia-smi

Fri May 27 05:52:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    39W / 250W |  16157MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install -q "monai[tqdm, nibabel, qdown, ignite]" "itk" "itkwidgets"

     |████████████████████████████████| 990 kB 2.7 MB/s 
     |████████████████████████████████| 3.4 MB 51.5 MB/s 
     |████████████████████████████████| 95.3 MB 34 kB/s 
     |████████████████████████████████| 16.6 MB 27.5 MB/s 
     |████████████████████████████████| 54.5 MB 1.3 MB/s 
     |████████████████████████████████| 20.3 MB 11.7 MB/s 
     |████████████████████████████████| 15.0 MB 33.2 MB/s 
     |████████████████████████████████| 70.6 MB 7.7 kB/s 
     |████████████████████████████████| 862 kB 56.4 MB/s 
     |████████████████████████████████| 2.5 MB 41.0 MB/s 
     |████████████████████████████████| 508 kB 53.5 MB/s 
     |████████████████████████████████| 271 kB 62.3 MB/s 
     |████████████████████████████████| 11.2 MB 44.6 MB/s 
     |████████████████████████████████| 944 kB 57.3 MB/s 
     |████████████████████████████████| 259 kB 60.4 MB/s 


In [2]:
import os
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/Deep Learning/")

Mounted at /content/drive


## Import Packages

In [3]:
import logging
import os
import sys
import tempfile
from glob import glob
import time

import torch
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference, SimpleInferer
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandAdjustContrastd,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    AsChannelFirstd,
    AsChannelLast,
    Resized,
    RandScaleCropd,
    RandRotated,
    Rotated,
    SaveImage,
)
from monai.visualize import plot_2d_or_3d_image

## Check Monai Configurations

In [4]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 0.9.1
Numpy version: 1.21.6
Pytorch version: 1.12.0+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 356d2d2f41b473f588899d705bbc682308cee52c
MONAI __file__: /usr/local/lib/python3.7/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 3.0.2
scikit-image version: 0.18.3
Pillow version: 7.1.2
Tensorboard version: 2.8.0
gdown version: 4.4.0
TorchVision version: 0.13.0+cu113
tqdm version: 4.64.0
lmdb version: 0.99
psutil version: 5.4.8
pandas version: 1.3.5
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Process VGH Data

In [5]:
data_path = "/content/drive/MyDrive/Deep Learning/SEG_Train_Datasets/dataset/"

-obtain train data list

In [6]:
tempdir = data_path + "train/image/"
train_images = sorted(glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "train/msk_img/"
train_segs = sorted(glob(os.path.join(tempdir, "*.png")))
print(f" {len(train_images)} train_images and {len(train_segs)} train_segs")
train_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[:], train_segs[:])]

 842 train_images and 842 train_segs


-obtain validation data list

In [7]:
tempdir = data_path + "validation/image/"
valid_images = sorted(glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "validation/msk_img/"
valid_segs = sorted(glob(os.path.join(tempdir, "*.png")))
print(f" {len(valid_images)} valid_images and {len(valid_segs)} valid_segs")

val_files = [{"img": img, "seg": seg} for img, seg in zip(valid_images[:], valid_segs[:])]

 105 valid_images and 105 valid_segs


## Define Transform for image and Segmentation

In [8]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        #RandScaleCropd(keys=["img", "seg"],roi_scale=0.5,max_roi_scale=1.5),
        #RandRotated(keys=["img", "seg"],range_x=3.14),
        RandAdjustContrastd(keys=["img"], prob=0.1, gamma=(0.5, 4.5)),
        #RandCropByPosNegLabeld(
        #    keys=["img", "seg"], label_key="seg", spatial_size=[800, 800], pos=1, neg=1, num_samples=4
        #),
        # RandRotate90d(keys=["img", "seg"], prob=0.5),
        Resized(keys=["img", "seg"], spatial_size=[1600, 800]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        Resized(keys=["img", "seg"], spatial_size=[1696, 928]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)

## Check and Visualize the Transform Results

In [9]:
# define dataset, data loader
check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

In [10]:
batch = 4
check_loader = DataLoader(check_ds, batch_size=batch, num_workers=12, collate_fn=list_data_collate)
check_data = monai.utils.misc.first(check_loader)
print(check_data["img"].shape, check_data["seg"].shape)


# import matplotlib.pyplot as plt

# plt.figure("visualize",(16,64))
# for i in range(batch):
#     plt.subplot(8,2,2*i+1)    
#     plt.imshow(check_data["img"][i].permute(1,2,0))
#     plt.subplot(8,2,2*i+2)
#     plt.imshow(check_data["seg"][i].permute(1,2,0).squeeze())

This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


(4, 3, 1600, 800) (4, 1, 1600, 800)


## Create DataLoader for Train and Validation Data

In [11]:
# create a training data loader
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)

# create a validation data loader
val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=4, collate_fn=list_data_collate)

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


## Define metric and post- processing

In [12]:
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
post_trans_label = Compose([EnsureType(), AsDiscrete(threshold=0.5)])

## Built Model

In [13]:
# create UNet, DiceLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.DynUNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    kernel_size=(3,3,3,3),
    strides=(1,2,2,2),
    upsample_kernel_size=(2,2,2,2),
    res_block=True,
    trans_bias=True,
).to(device)


loss_function = monai.losses.DiceLoss(sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5)

In [14]:
!pip install netron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 3.2 MB/s 


In [17]:
import netron

onnx_path = "model.onnx"
input_dummy = torch.randn(4,3,1600,800).to(device)
model.eval()

torch.onnx.export(model , input_dummy , onnx_path)

ONNX export mode is set to inference mode, but operator instance_norm is set to training  mode. The operators will be exported in training , as specified by the functional operator.


In [18]:
netron.start("model.onnx")

Serving 'model.onnx' at http://localhost:8081


('localhost', 8081)

## Create Visualize Function

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

## Define Training Parameters and Start training

In [ ]:
model.load_state_dict(torch.load("best_metric_model_segmentation2d_dict0525.pth"))
model.eval()

DynUNet(
  (input_block): UnetResBlock(
    (conv1): Convolution(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (conv3): Convolution(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (norm2): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (norm3): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  )
  (downsamples): ModuleList(
    (0): UnetResBlock(
      (conv1): Convolution(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (conv2): Convolution(
        (conv): Conv2d(64, 64, kernel_size=

In [ ]:
#### start a typical PyTorch training
total_epochs = 40
val_interval = 1
best_metric = 0
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
writer = SummaryWriter()
for epoch in range(total_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{total_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        
        outputs=[post_trans(i) for i in decollate_batch(outputs)]
        labels=[post_trans_label(i) for i in decollate_batch(labels)] 
        dice_metric(y_pred=outputs, y=labels)
        
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        # print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    metric = dice_metric.aggregate().item()
    dice_metric.reset()
    #print("current training dice score: {:.4f} ".format(metric))
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    local_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    print(f"{local_time} epoch {epoch + 1} average loss: {epoch_loss:.4f} dice score:{metric}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            show_val = True
            for val_data in val_loader:
                val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                roi_size = (1600, 800)
                sw_batch_size = 4
                val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model) 
                
                #if show_val:
                #    visualize( 
                #        image=val_images[0].cpu().permute(1,2,0), 
                #        ground_truth_mask=val_labels[0].cpu().permute(1,2,0), 
                #        predicted_mask=val_outputs[0].cpu().permute(1,2,0)
                #    )        
                
                
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                
                val_labels = [post_trans_label(i) for i in decollate_batch(val_labels)]
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)

                # if show_val:
                #     visualize( 
                #         image=val_images[0].cpu().permute(1,2,0), 
                #         ground_truth_mask=val_labels[0].cpu().permute(1,2,0).squeeze(), 
                #         predicted_mask=val_outputs[0].cpu().permute(1,2,0).squeeze()
                #     )                                      
                
                show_val = False
                

            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()            
            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model_segmentation2d_dict0527.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current val mean dice score: {:.4f} best val mean dice score: {:.4f} at epoch {}".format(
                    epoch + 1, metric, best_metric, best_metric_epoch
                )
            )
            writer.add_scalar("val_mean_dice score", metric, epoch + 1)
            # plot the last model output as GIF image in TensorBoard with the corresponding image and label
            plot_2d_or_3d_image(val_images, epoch + 1, writer, index=0, tag="image")
            plot_2d_or_3d_image(val_labels, epoch + 1, writer, index=0, tag="label")
            plot_2d_or_3d_image(val_outputs, epoch + 1, writer, index=0, tag="output")
            

print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()


torch.save(model.state_dict(), "Final_model_40_epoches_segmentation2d_dict0527.pth")

----------
epoch 1/40


RuntimeError: ignored

-obtain test datalist

In [ ]:
# Load testing files
tempdir = data_path + "test/image/"
test_images = sorted(glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "test/msk_img/"
test_segs = sorted(glob(os.path.join(tempdir, "*.png")))

print(f" {len(test_images)} test_images and {len(test_segs)} test_segs")
# print(f" {len(test_images)} test_images")

test_files = [{"img": img, "seg": seg} for img, seg in zip(test_images[:], test_segs[:])]

 106 test_images and 106 test_segs


In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),

        ScaleIntensityd(keys=["img", "seg"]),
        #Resized(keys=["img", "seg"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
test_ds = monai.data.Dataset(data=test_files, transform=test_transforms)

In [ ]:
test_loader = DataLoader(test_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate)

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


In [ ]:
model.load_state_dict(torch.load("best_metric_model_segmentation2d_dict.pth"))
model.eval()

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = test_data["img"].to(device), test_data["seg"].to(device)
        # define sliding window size and batch size for windows inference
        roi_size = (800, 800)
        sw_batch_size = 4
        test_outputs = sliding_window_inference(test_images, roi_size, sw_batch_size, model)

        # visualize( 
        #     image=test_images[0].cpu().permute(1,2,0), 
        #     ground_truth_mask=test_labels[0].cpu().permute(1,2,0).squeeze(), 
        #     predicted_mask=test_outputs[0].squeeze().cpu().numpy().round()
        # )           
        # saverGT(test_labels[0].cpu())
        # saverIM(test_images[0].cpu())        
        # saverPD(test_outputs[0].cpu())
        
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]
        test_labels = [post_trans(i) for i in decollate_batch(test_labels)]
        
        
                
        # compute metric for current iteration
        dice_metric(y_pred=test_outputs, y=test_labels)
        #for test_output in test_outputs:            
        #    saver(test_output*255)
    # aggregate the final mean dice result    
    print("evaluation metric:", dice_metric.aggregate().item())
    # reset the status
    dice_metric.reset()

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


evaluation metric: 0.04734683409333229


## Prediction on Public Image

-obtain data list

In [ ]:
public_path = "/content/drive/MyDrive/Deep Learning/"

In [ ]:
# Load testing files
tempdir = public_path + "Public_Image/"
pred_images = sorted(glob(os.path.join(tempdir, "*.jpg")))

#tempdir = data_path + "Valid_Masks/"
#test_segs = sorted(glob(os.path.join(tempdir, "*.png")))

#print(f" {len(test_images)} test_images and {len(test_segs)} test_segs")
print(f" {len(pred_images)} pred_images")

pred_files = [{"img": img} for img in zip(pred_images[:])]

 131 pred_images


## Define Transform for Prediction data

In [ ]:
# define transforms for image and segmentation
pred_transforms = Compose(
    [
        LoadImaged(keys=["img"]),
        
        #Rotated(keys=["img"], angle=90),
        AsChannelFirstd(keys=["img"]),

        ScaleIntensityd(keys=["img"]),
        #Resized(keys=["img", "seg"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"]),
    ]
)
pred_ds = monai.data.Dataset(data=pred_files, transform=pred_transforms)


post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])

## Save Images

In [ ]:
pred_loader = DataLoader(pred_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate)
saverPD = SaveImage(output_dir="./prediction0526_2", output_ext=".png", output_postfix="PD",scale=255,separate_folder=False)

## Load Model

In [ ]:
model.load_state_dict(torch.load("best_metric_model_segmentation2d_dict0526_2.pth" ))
model.eval()

DynUNet(
  (input_block): UnetResBlock(
    (conv1): Convolution(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (conv3): Convolution(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (norm2): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (norm3): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  )
  (downsamples): ModuleList(
    (0): UnetResBlock(
      (conv1): Convolution(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (conv2): Convolution(
        (conv): Conv2d(64, 64, kernel_size=

In [ ]:
# import shutil
# shutil.rmtree("/content/drive/MyDrive/Deep Learning/prediction0525")

In [ ]:
with torch.no_grad():
    for test_data in pred_loader:
        test_images= test_data["img"].to(device)
        # define sliding window size and batch size for windows inference
        roi_size = (1696, 928)
        sw_batch_size = 2
        test_outputs = sliding_window_inference(test_images, roi_size, sw_batch_size, model)

        # visualize( 
        #     image=test_images[0].cpu().permute(1,2,0),             
        #     predicted_mask=test_outputs[0].squeeze().cpu().numpy().round()
        # )                   
        #saverIM(test_images[0].cpu())        
        saverPD(test_outputs[0].cpu())
        
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]  

file written: prediction0526_2/0_PD.png.
file written: prediction0526_2/1_PD.png.
file written: prediction0526_2/2_PD.png.
file written: prediction0526_2/3_PD.png.
file written: prediction0526_2/4_PD.png.
file written: prediction0526_2/5_PD.png.
file written: prediction0526_2/6_PD.png.
file written: prediction0526_2/7_PD.png.
file written: prediction0526_2/8_PD.png.
file written: prediction0526_2/9_PD.png.
file written: prediction0526_2/10_PD.png.
file written: prediction0526_2/11_PD.png.
file written: prediction0526_2/12_PD.png.
file written: prediction0526_2/13_PD.png.
file written: prediction0526_2/14_PD.png.
file written: prediction0526_2/15_PD.png.
file written: prediction0526_2/16_PD.png.
file written: prediction0526_2/17_PD.png.
file written: prediction0526_2/18_PD.png.
file written: prediction0526_2/19_PD.png.
file written: prediction0526_2/20_PD.png.
file written: prediction0526_2/21_PD.png.
file written: prediction0526_2/22_PD.png.
file written: prediction0526_2/23_PD.png.
fi

In [ ]:
import glob
prediction_path = glob.glob("/content/drive/MyDrive/Deep Learning/prediction0525/*")

image_name = os.path.basename(prediction_path[0])
new_name = image_name.split("_")[0].rjust(8,"0")

In [ ]:
import glob
import shutil
prediction_path = glob.glob("/content/drive/MyDrive/Deep Learning/prediction0526_2/*")

dest_path = "/content/drive/MyDrive/Deep Learning/prediction0526_2/0526_2/"
os.makedirs(dest_path)
for path in prediction_path:
  image_name = os.path.basename(path)
  new_name = image_name.split("_")[0].rjust(8, "0")
  # new_name = new_name.rjust(8, "0")
  new_name = "Public_" + new_name + ".png"
  new_path = os.path.join(dest_path, new_name)
  shutil.copy(path , new_path)